In [1]:
# standard imports
import numpy as np
import pandas as pd
# visualization
import matplotlib.pyplot as plt
import seaborn as sns
# stats
from scipy import stats
# modeling
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, LassoLars
from sklearn.metrics import explained_variance_score
# notebook formatting
from pprint import pprint
import warnings
warnings.filterwarnings("ignore")


# acquire dats
from env import user, password, host
import os

In [2]:
def acquire_from_sql(db, query, csv_name):
    ''' Acquires data from SQL using env imports and returns 
        a pandas data frame. The acquired dataframe is saved 
        as a new csv file.'''
    url = f'mysql+pymysql://{user}:{password}@{host}/{db}'
    if os.path.isfile(f'{csv_name}.csv'):
        df = pd.read_csv(f'{csv_name}.csv', index_col=0)
    else:
        df = pd.read_sql(query, url)
        df.to_csv(f'{csv_name}.csv')
    return df

In [3]:
query = '''SELECT *, logs.cohort_id AS cohort_id
            FROM logs
            LEFT JOIN cohorts ON logs.cohort_id = cohorts.id;'''

df = acquire_from_sql('curriculum_logs', query, 'curriculum_logs')
df.head()

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id,cohort_id.1
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0,8.0
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0,8.0
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0,8.0
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0,8.0
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,22.0,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,NaN,2.0,22.0


In [4]:
# summarize data/ inital glace at data
print('_'*50)
print(f'Shape: \n{df.shape}')
print('_'*50)
print(f'Stats: \n{df.describe().T}')
print('_'*50)
print('Info: ')
print(df.info())
print('_'*50)
print(f'Data Types: \n{df.dtypes}')
print('_'*50)
print(f'Null Values: \n{df.isnull().sum()}')
print('_'*50)
print(f'NA Values: \n{df.isna().sum()}')
print('_'*50)
print(f'Unique Value Count: \n{df.nunique()}')
print('_'*50)
print(f'Columns: \n{df.columns}')
print('_'*50)
print(f'Column Value Counts: \n{df.columns.value_counts(dropna=False)}')
print('_'*50)

__________________________________________________
Shape: 
(900223, 16)
__________________________________________________
Stats: 
                count        mean         std  min    25%    50%    75%    max
user_id      900223.0  458.825707  249.296767  1.0  269.0  475.0  660.0  981.0
cohort_id    847330.0   48.501049   32.795482  1.0   28.0   33.0   57.0  139.0
id           847330.0   48.501049   32.795482  1.0   28.0   33.0   57.0  139.0
deleted_at        0.0         NaN         NaN  NaN    NaN    NaN    NaN    NaN
program_id   847330.0    2.086004    0.388231  1.0    2.0    2.0    2.0    4.0
cohort_id.1  847330.0   48.501049   32.795482  1.0   28.0   33.0   57.0  139.0
__________________________________________________
Info: 
<class 'pandas.core.frame.DataFrame'>
Int64Index: 900223 entries, 0 to 900222
Data columns (total 16 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   date         900223 non-null  object 
 1   time         

In [5]:
# drop specified columns
df = df.drop(columns=['slack', 'id', 'cohort_id.1', 'deleted_at'])

In [18]:
# (number of null values/ total number of value)*100
# = percent of data that has null values
(52893/900223)* 100

5.875544170722144

In [21]:
# sum of boolean mask of null values
null_counts = df.isnull().sum(axis=1)

# filter the DataFrame to include only rows with multiple null values
rows_with_nulls = df[null_counts == 7]
rows_with_nulls

,date,time,path,user_id,cohort_id,ip,name,start_date,end_date,created_at,...,392,393,394,395,396,397,398,399,400,401
411,2018-01-26,16:46:16,/,48,NaN,97.105.19.61,NaN,NaN,NaN,NaN,...,48,48,48,48,48,48,48,48,48,48
412,2018-01-26,16:46:24,spring/extra-features/form-validation,48,NaN,97.105.19.61,NaN,NaN,NaN,NaN,...,48,48,48,48,48,48,48,48,48,48
425,2018-01-26,17:54:24,/,48,NaN,97.105.19.61,NaN,NaN,NaN,NaN,...,48,48,48,48,48,48,48,48,48,48
435,2018-01-26,18:32:03,/,48,NaN,97.105.19.61,NaN,NaN,NaN,NaN,...,48,48,48,48,48,48,48,48,48,48
436,2018-01-26,18:32:17,mysql/relationships/joins,48,NaN,97.105.19.61,NaN,NaN,NaN,NaN,...,48,48,48,48,48,48,48,48,48,48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899897,2021-04-21,12:49:00,javascript-ii,717,NaN,136.50.102.126,NaN,NaN,NaN,NaN,...,48,48,48,48,48,48,48,48,48,48
899898,2021-04-21,12:49:02,javascript-ii/es6,717,NaN,136.50.102.126,NaN,NaN,NaN,NaN,...,48,48,48,48,48,48,48,48,48,48
899899,2021-04-21,12:51:27,javascript-ii/map-filter-reduce,717,NaN,136.50.102.126,NaN,NaN,NaN,NaN,...,48,48,48,48,48,48,48,48,48,48
899900,2021-04-21,12:52:37,javascript-ii/promises,717,NaN,136.50.102.126,NaN,NaN,NaN,NaN,...,48,48,48,48,48,48,48,48,48,48


In [22]:
data = pd.crosstab(df.cohort_id, df.user_id)

In [23]:
data

user_id,1,2,3,4,5,6,7,8,9,10,...,972,973,974,975,976,977,978,979,980,981
cohort_id,,,,,,,,,,,,,,,,,,,,,
1.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8.0,1617,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
